## 1. impl Trait 처리 

### 타입을 처리하는 함수 정의 

In [2]:
use std::any::type_name;

fn type_of<T>(_: T) -> &'static str {
    type_name::<T>()
}

## 1-1 impl 트레이트 지정하기

- 함수 매개변수와 반환타입으로 지정이 가능 
- 트레이트가 구현된 매개변수를 전달을 받아서 처리할 수 있다. 

In [3]:
fn copy(_item: impl Copy) {
    println!("Copy");
}

fn clone(_item: impl Clone) {
    println!("Clone");
}

In [4]:
fn main() {
    let num = 1;
    copy(num);
    clone(num);

    let string = String::from("Hello");
    clone(string);
    // copy(string); // 🤯
}
main();

Copy
Clone
Clone


## 1-2  함수 자료형도 구현 트레이트 처리 

### 구현 트레이트로 처리하기 

In [5]:
fn f(g: impl FnOnce() -> String) {
    println!("{}", g());
}

In [6]:
fn main() {
    let mut s = String::from("foo");
    let t = String::from("bar");

    f(|| {s += &t;s});
}

main();

foobar


### 구현 트레이트 처리시 예외 발생 

- 왜냐하면 구현 트레이트는 오직 함수와 메서드만 처리가 가능
- 클로저로 처리된 경우는 예외발생 

In [7]:
pub fn make_quadratic(a: f64, b: f64, c: f64) -> impl Fn(f64) -> f64 {
    move |x| a*x*x + b*x + c
}

### 클로저가 들어오면 예외  

In [8]:
fn main() { 
    let quad_fn: impl Fn(f64) -> f64 = make_quadratic(5.0, 4.0, 3.0);
           //error: `impl Trait` only allowed in function and inherent method return types
    println!("{}", type_of(&quad_fn));
}

main();

Error: `impl Trait` only allowed in function and inherent method return types, not in variable bindings

### 그래서 반환결과를 처리 결과로 변환해야 함 

In [9]:
fn main() { 
    let quad_fn = make_quadratic(5.0, 4.0, 3.0);
    
    println!("{}", type_of(&quad_fn));
    
    println!("{}", call_make_quadratic(10.0));
}


pub fn call_make_quadratic(x: f64) -> f64 {
    let quad_fn = make_quadratic(5.0, 4.0, 3.0);
    quad_fn(x) 
}

main();

&ctx::make_quadratic::{{closure}}
543


## 1-3 구현트레이트와 트레이트 객체 혼용

In [11]:
pub fn make_quadratic_box(a: f64, b: f64, c: f64) -> Box<dyn Fn(f64) -> f64> {
    Box::new(make_quadratic(a, b, c))
}

pub fn make_quadratic_no_capture() -> impl Fn(f64) -> f64 {
    |x| 42.0*x*x + 84.0*x + 0.0
}

pub fn make_quadratic_box_no_capture() -> Box<dyn Fn(f64) -> f64> {
    Box::new(make_quadratic_no_capture())
}

In [12]:
fn main() {
    let a = 100.23f64;
    let b = make_quadratic_box(a,a,a);
    println!("{}", b(a));
    
    let c = make_quadratic_no_capture();
    println!("{}", c(a));
    let d = make_quadratic_box_no_capture();
    println!("{}", d(a));
}

main();

1017062.1650670001
430353.5418
430353.5418
